In [14]:
import requests
from requests.auth import HTTPDigestAuth
class AxisCamera:
    def __init__(self, ip, uname, passwd):
        self.ip = ip
        self.auth = HTTPDigestAuth(uname, passwd)

    def ptz(self, args):
        url=f'http://{self.ip}/axis-cgi/com/ptz.cgi?{args}'
        print(url)
        return requests.get(url, auth=self.auth, timeout=1).content

    def setposition(self, pos):
        if 0:
            self.ptz(f'pan={pos["pan"]}')
            time.sleep(0.1)
            self.ptz(f'tilt={pos["tilt"]}')
            time.sleep(0.1)
            self.ptz(f'zoom={pos["zoom"]}')
        self.ptz(f'pan={pos["pan"]}&tilt={pos["tilt"]}&zoom={pos["zoom"]}')

    def position(self):
        s = self.ptz('query=position').decode('utf8')
        D = {}
        for line in s.split():
            k, v = line.split('=')
            if v in ['on', 'off']:
                # D[k]=(v=='on')
                D[k] = v
            else:
                try:
                    D[k] = int(v)
                except:
                    D[k] = float(v)
        return D

    def savepos(self, fname):
        open(fname, 'w').write(json.dumps(self.position()))

    def loadposthread(self, pos):
        if simulation:
            return True
        for i in range(3):
            try:
                self.setposition(pos)
                break
            except:                
                print('set position', 'error redo')
                time.sleep(0.05)

    def loadpos(self, fname):
        """load pos from json file"""
        if simulation:
            return True
        pos = json.loads(open(fname).read())
        thread = Thread(target=self.loadposthread, args=(pos,), daemon=True)
        thread.start()

    def loadpostxt(self, fname, tiltoffset=None, forcezoom=None):
        """load pos from text file"""
        if simulation:
            return True
        pos = dict()
        for line in open(fname).read().strip().split():
            k, v = line.split('=')
            pos[k] = v
        if tiltoffset is not None:
            pos['tilt']=float(pos['tilt'])+tiltoffset
        if forcezoom is not None:
            pos['zoom']=forcezoom
        thread = Thread(target=self.loadposthread, args=(pos,), daemon=True)
        thread.start()
    
    def atpostxt(self, fname, tiltoffset=None, forcezoom=None):
        """load pos from text file"""
        if simulation:
            return True
        pos = dict()
        for line in open(fname).read().strip().split():
            k, v = line.split('=')
            pos[k] = float(v)
        if tiltoffset is not None:
            pos['tilt']=float(pos['tilt'])+tiltoffset
        if forcezoom is not None:
            pos['zoom']=float(forcezoom)
        try:
            postarget=self.position()
            #gd21
            return min((postarget['pan']-pos['pan'])%360,(pos['pan']-postarget['pan'])%360)<=0.1 and abs(postarget['tilt']-pos['tilt'])<=0.1 and abs(postarget['zoom']-pos['zoom'])<=0.1
        except:
            return False
    def loadposstr(self, s):
        """load pos from string"""
        if simulation:
            return True
        pos = dict()
        for line in s.strip().split():
            k, v = line.split('=')
            pos[k] = v
        thread = Thread(target=self.loadposthread, args=(pos,), daemon=True)
        thread.start()

    def positiontxt(self):
        if simulation:
            return 'pan=0\ntilt=0\nzoom=0'
        s=(self.ptz('query=position').decode('utf8'))
        print(s)
        return(s)

    def snapshotthread(self, filename,delay):
        try:
            time.sleep(delay)
            image=requests.get(f'http://{self.ip}/axis-cgi/jpg/image.cgi?resolution=1280x720', auth=self.auth, timeout=1).content
            open(filename,'wb').write(image)
        except:
            pass

    def snapshot(self,filename,delay=0):
        DIR=os.path.dirname(filename)
        if DIR!='':
            os.makedirs(DIR,exist_ok=True)
        thread = Thread(target=self.snapshotthread, args=(filename,delay), daemon=True)
        thread.start()

    def snapshotimage(self):
        try:
            image=requests.get(f'http://{self.ip}/axis-cgi/jpg/image.cgi?resolution=1280x720', auth=self.auth, timeout=1).content
            return image
        except:
            return None
            pass
        
    def nonptzautofocusthread(self):
        return requests.get(f'http://{self.ip}/axis-cgi/opticssetup.cgi?autofocus=perform', auth=self.auth, timeout=20).content

    def nonptzautofocus(self):
        thread = Thread(target=self.nonptzautofocusthread, daemon=True)
        thread.start()

In [15]:
cam=AxisCamera('10.140.66.84','root','P456YC_pptcam')

In [16]:
cam.position()

http://10.140.66.84/axis-cgi/com/ptz.cgi?query=position


ConnectTimeout: HTTPConnectionPool(host='10.140.66.84', port=80): Max retries exceeded with url: /axis-cgi/com/ptz.cgi?query=position (Caused by ConnectTimeoutError(<urllib3.connection.HTTPConnection object at 0x7fac5fcf1e10>, 'Connection to 10.140.66.84 timed out. (connect timeout=1)'))

In [34]:
cam.setposition({'pan':200,'tilt':-144,'zoom':10000})

In [61]:
import time
import random
def randompos():
    return {'pan':random.randint(-360,360),'tilt':random.randint(-90,90),'zoom':random.randint(1,7930)}
def atpos(pos,postarget):
    return min((postarget['pan']-pos['pan'])%360,(pos['pan']-postarget['pan'])%360)<=0.1 and abs(postarget['tilt']-pos['tilt'])<=0.1 and abs(postarget['zoom']-pos['zoom'])<=1
f=open('log.txt','w')
for i in range(1000):
    T=time.time()
    postarget=randompos()
    cam.setposition(postarget)
    while 1:
        time.sleep(0.1)
        elapse=time.time()-T        
        pos=cam.position()
        print(i,pos["pan"],pos["tilt"],pos["zoom"],postarget["pan"],postarget["tilt"],postarget["zoom"],elapse,file=open('log.txt','a'))                    
        if atpos(pos,postarget) or elapse>3:
            break
    time.sleep(1)

KeyboardInterrupt: 

In [58]:
print(pos,postarget)


{'pan': 111.99, 'tilt': -0.0, 'zoom': 7441, 'focus': 4151, 'brightness': 5000, 'autofocus': 'on', 'autoiris': 'on'} {'pan': 112, 'tilt': 84, 'zoom': 7441}
